In [1]:
from time import time
import numpy as np
from numpy.random import randint
from numba import jit, vectorize, int64

## Basic Python Loop Speed-Up

In [2]:
def insertion_sort(arr):
    for i in range(len(arr)):
        cursor = arr[i]
        pos = i
        while pos > 0 and arr[pos - 1] > cursor:
            # Swap the number down the list
            arr[pos] = arr[pos - 1]
            pos = pos - 1
        # Break and do the final swap
        arr[pos] = cursor
    return arr


@jit(nopython=True)
def insertion_sort_numba(arr):
    for i in range(len(arr)):
        cursor = arr[i]
        pos = i
        while pos > 0 and arr[pos - 1] > cursor:
            # Swap the number down the list
            arr[pos] = arr[pos - 1]
            pos = pos - 1
        # Break and do the final swap
        arr[pos] = cursor
    return arr

def insertion_experiment(num_loops=50,len_of_list=100000):
    arr = randint(0, 3000, size=len_of_list).tolist()
    start = time()
    for i in range(num_loops):
        insertion_sort(arr)
    run_time = time() - start
    print("No Numba: Average time for {} elements = {:10.5f}s".format(len_of_list, run_time / num_loops))

    start = time()
    for i in range(num_loops):
        insertion_sort_numba(arr)
    run_time_numba = time() - start
    print("Numba: Average time for {} elements = {:10.5f}s".format(len_of_list, run_time_numba / num_loops))
    
    print("Numba speeds up by {:5.0f} times".format(run_time/run_time_numba))


In [13]:
insertion_experiment(num_loops=30,len_of_list=50000)

No Numba: Average time for 50000 elements =    5.17141s
Numba: Average time for 50000 elements =    0.00257s
Numba speeds up by  2015 times


## Numpy Operation Speed-Up

In [3]:
def add_arrays(img1,img2,img3):
    return np.square(img1 + img2 + img3)

@vectorize([int64(int64,int64,int64)],target="parallel")
def add_arrays_numba_parallel(img1,img2,img3):
    return np.square(img1 + img2 + img3)

@vectorize([int64(int64,int64,int64)],target="cpu")
def add_arrays_numba_cpu(img1,img2,img3):
    return np.square(img1 + img2 + img3)

def add_arrays_experiment(N,num_loops):
    img1 = np.ones((N, N), np.int64) * 5
    img2 = np.ones((N, N), np.int64) * 10
    img3 = np.ones((N, N), np.int64) * 15
    
    print("++Add & Square========>")
    
    print("Data Size: {} MB".format(img1.nbytes/(10**6)))
    start = time()
    for i in range(num_loops):
        add_arrays(img1,img2,img3)
    run_time = time() - start
    print("No Numba: Average time for {}X{} images = {:10.5f}s".format(N, N, run_time / num_loops))
    
    start = time()
    for i in range(num_loops):
        add_arrays_numba_parallel(img1,img2,img3)
    run_time_numba_parallel = time() - start
    print("Numba Parallel: Average time for {}X{} images = {:10.5f}s".format(N, N, run_time_numba_parallel / num_loops))
    
    start = time()
    for i in range(num_loops):
        add_arrays_numba_cpu(img1,img2,img3)
    run_time_numba_cpu = time() - start
    print("Numba CPU: Average time for {}X{} images = {:10.5f}s".format(N, N, run_time_numba_cpu / num_loops))
    
    print("Numba Parallel speeds up by {:5.5f} times".format(run_time/run_time_numba_parallel))
    print("Numba CPU speeds up by {:5.5f} times".format(run_time/run_time_numba_cpu))

In [6]:
add_arrays_experiment(5000,30)

++Add & Square========>
Data Size: 200.0 MB
No Numba: Average time for 5000X5000 images =    0.31577s
Numba Parallel: Average time for 5000X5000 images =    0.09357s
Numba CPU: Average time for 5000X5000 images =    0.14734s
Numba Parallel speeds up by 3.37466 times
Numba CPU speeds up by 2.14318 times


In [7]:
add_arrays_experiment(1000,30)

++Add & Square========>
Data Size: 8.0 MB
No Numba: Average time for 1000X1000 images =    0.00756s
Numba Parallel: Average time for 1000X1000 images =    0.00346s
Numba CPU: Average time for 1000X1000 images =    0.00338s
Numba Parallel speeds up by 2.18825 times
Numba CPU speeds up by 2.23612 times


In [9]:
add_arrays_experiment(900,30)

++Add & Square========>
Data Size: 6.48 MB
No Numba: Average time for 900X900 images =    0.00628s
Numba Parallel: Average time for 900X900 images =    0.00315s
Numba CPU: Average time for 900X900 images =    0.00294s
Numba Parallel speeds up by 1.99539 times
Numba CPU speeds up by 2.13672 times


CPU outperforms Parallel model on small data; Parallel model out performs CPU on large data